# Part 1: DNA sequences

This homework follows some of the results in importance sampling found in the paper included in this repository.

The main idea is to model DNA sequences as a Markov Chain and to determine the probability of finding particular motifs within the pattern.

DNA is made up of sequences of four nucleotides, ATCG (with complenetary pairs TAGC, respectively).  We will call the set $\{A,T,C,G\}$ an alphabet and denote the set as $\chi$.

Denote a sequence as $\mathbf{s}_n\in\chi^n$, and let $s_i$ be the $i$th term in the sequence.  For example, if $n=4$, let $\mathbf{s}_4 = AACT$, and then $s_3=C$.  Suppose the probability of finding a particular sequence of length $n$ is equal to 

$q_s(\mathbf{s}) = \pi(s_1) \prod_{i=2}^n\sigma(s_{i-1}s_{i}),$

where $\sigma(ab)$ is the transition probability of a markov chain defined as $P(s_{i+1}=b | s_{i}=a)$, and $\pi$ is the steady state distribution of the resulting transition matrix. (Bonus: why is the above a distribution? i.e. normalized?)

In the following, take the transition matrix (found in Section 5.2) to be 

$T = \begin{pmatrix}
.3577 & .1752 & .1853 & .2818\\
.2381 & .1943 & .1905 & .3771\\
.3256 & .2056 & .1592 & .3096\\
.2992 & .2180 & .2039 & .2789\\
\end{pmatrix}$

Where the ordering of the rows/columns corresponds to ATCG.

**Question 1.** Determine the steady state distribution, $\pi$ of this matrix.  What does your answer tell you about the symmetry or asymmetry of DNA sequences (given that every A we find has a corresponding T on the opposite chain, and vice versa; similarly for C to G)? 
**Question 2.** Determine if the markov chain respects detailed balance (i.e. is it reversible?)  What does the model predict about the physical properties of DNA sequences?  Is this consistent with your previous conclusion?  
**Task 3.** Write a function called `generateDNASequence` that takes in a length, $n$, and draws a sample of length $n$ from the described probability distribution.

# Part 2: Word detection with direct Monte Carlo
When considering sequences of DNA, we often wish to know about the prevelance of certain genes or features.

One tool to investigate "word" (i.e. gene prevelance) is to use a "Position-Specific Weight Matrix" (PSWM) which works as follows:

Suppose we have a particular word pattern, or distribution of word patterns of length $m$ and we wish to determine how close given sequence of length $m$ matches the pattern.

The examples used in the paper comes from another work and specifies a particular PSWM for the transcription factor of yeast (with $m=12$) as 

$\begin{pmatrix}
4 & 0 & 4 & 1 & 1 & 4 & 0 & 0 & 0 & 0 & 0 & 2\\
0 & 3 & 2 & 3 & 2 & 1 & 5 & 0 & 0 & 7 & 0 & 0\\
1 & 2 & 1 & 1 & 3 & 2 & 0 & 0 & 7 & 0 & 0 & 0\\
2 & 2 & 0 & 2 & 1 & 0 & 2 & 7 & 0 & 0 & 7 & 5\\
\end{pmatrix}$

again using ATCG ordering (slightly different from the paper).

We say that the score of a sequence of length $m$ has a score, $S(\mathbf{v}_m) = \sum_{i=1}^m w_i(v_i)$ where $w_i(v_i)$ is the value of the $i$th column in the row corresponding to $v_i$.  Thus, in the above example, the subsequence of length 12, AAAAAAAAAAAA would simply have a score equal to the sum of the first row ($= 4 + 0 + 4 + 1 + 1 + 4 + 0 + 0 + 0 + 0 + 0 + 2$), and if we had AAAAAAAAAAAG, the last value in the sum would change from a 2 to a 5 (since the last column has G value of 5).

**Question 4.** Use direct Monte Carlo (i.e. simply sample from the measure as you have done above) by taking 1000 trials; estimate the probability that a DNA segment of length 200 has at least one subsegment of length 12 with a score of larger than (i) 9, (ii) 10, (iii) 11 according to the PSWM

$P_{\text{simple}} = \begin{pmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
\end{pmatrix}$

Estimate the uncertainty (standard deviation) of your result (either analytically or numerically).

# Importance sampling on words

Continuing the example from above, again consider words $\mathbf{v}$ of length 12 using the simple PSWM we used above ($P_{\text{simple}}$).

Let's use imporance sampling to improve our estimates.  First, we want to sample words $\mathbf{v}$ of length 12 that are likely to have high scores.  To do this, let's sample from the distribution

$q_\theta(\mathbf{v}) \propto e^{\theta S(\mathbf{v})} \pi(v_1) \prod_{i=2}^{12} \sigma(v_{i-1}v_{i})/\Lambda(\theta) = e^{\theta w_1(v_1)} \pi(v_1) \prod_{i=2}^{12} e^{\theta w_i(v_i)} \sigma(v_{i-1}v_{i}) /\Lambda(\theta),$

where $\Lambda(\theta) = \sum_{\mathbf{v}}e^{\theta S(\mathbf{v})} \pi(v_1) \prod_{i=2}^{12} \sigma(v_{i-1}v_{i})$ is the (currently unknown) normalizing constant.

Notice that we can draw samples from $q_\theta$ by first drawing $v_1$ according to $\frac{\exp(\theta w_1(v_1))\pi(v_1)}{\sum_{v\in\chi}\exp(\theta w_1(v))\pi(v)}$, and then drawing $v_i$ according to $\frac{\exp(\theta w_i(v_i))\sigma(v_{i-1}v_i)}{\sum_{v\in\chi}\exp(\theta w_i(v))\sigma(v_{i-1}v)}$

Note that you will want to choose $\theta$ large enough so that you focus on words with high scores, but small enough so that you don't completely skew the distribution.

According to Algorithm A in the paper, we will randomly generate a sequence $\mathbf{s}_n$ by 
1. Randomly choosing an index $i$, such that $0 \leq i \leq n-m$ (in our case $m=12$)
2. Randomly sampling $\mathbf{v}$ from $q_\theta$
3. Generate the first $i$ elements of $\mathbf{s}_n$ by the method from Part 1 (i.e. start with the stationary distribution and continue with the Markov chain).
4. Insert $\mathbf{v}$ to be the next $m$ elements of $\mathbf{s}_n$
5. Continue to generate the remaining $n-(i+12)$ elements $\mathbf{s}_n$ with the Markov chain. 

**Question 5.** Determine (analytically) the probability of drawing a sequence $\mathbf{s}_n$ using algorithm A as above (for assistance see equation 2.3 in the paper)  
**Question 6.** Call the distribution from Question 5 $q_{\theta,s}$ and determine $q_{\theta,s}/q_s$, where $q_s$ is the distribution on sequences from Part 1 (again, for some help see equation 2.3 in the paper).  
**Question 7.** For a chosen $\theta$, code a function that approximates the normalization constant of $\Lambda(\theta)$ using importance sampling.  
**Question 8.** For a variety of $\theta>0$, determine the fraction of sequences of length 200, generated by Algorith A, that have a word with a score greater than or equal to 9, 10, and 11.  Do this by generating 1000 sequences of length 200 from Algorithm A.  What do you notice about the trend in $\theta$?  
**Question 9.** Choose a value of $\theta$ from your above experiments and use importance sampling to estimate the probability (and uncertainty) that a random sequence of DNA made up of 200 nucleotides will have a word with score greater than or equal to 9, 10, and 11 (again using 1000 different sequences of length 200). Are you concerned about bias introduced from approximating $\Lambda(\theta)$?  What could you do to reduce this bias?